# EDS Block Experimentation - Context Aware

This notebook works in both JSLab (Node.js) and browser environments.

## Environment Detection

- **JSLab/Node.js**: Sets up jsdom virtual DOM and helper functions
- **Browser (ipynb-viewer)**: Uses native browser APIs directly

## Important Notes

1. **In JSLab**: Run Cell 1 first to initialize the environment
2. **In Browser**: Cell 1 auto-detects and skips Node.js setup
3. **Pure EDS blocks** - Works best with vanilla JavaScript blocks
4. **Context-aware execution** - Automatically detects Node.js vs browser
5. **Visual output** - Generates live preview HTML with iframe controls
6. **Iframe previews** - Open `-live-preview.html` files for interactive testing

In [ ]:
// ============================================================================
// SETUP: Context-aware initialization
// Run this cell FIRST - works in both JSLab and Browser!
// ============================================================================

(async () => {
// Detect execution environment
const isNode = typeof process !== 'undefined' && process.versions && process.versions.node;
const isBrowser = typeof window !== 'undefined' && typeof document !== 'undefined';

console.log('Environment:', isNode ? 'Node.js (JSLab)' : 'Browser');

// ============================================================================
// NODE.JS SETUP (JSLab/Jupyter)
// ============================================================================
if (isNode) {
  const path = require('path');
  const helpersPath = path.resolve('./scripts/ipynb-helpers.js');
  const helpers = await import(helpersPath);
  
  // Initialize Node.js environment (jsdom + output directory)
  await helpers.setupNodeEnvironment();
  
  // Make helpers available globally
  global.loadBlockStyles = helpers.loadBlockStyles;
  global.testBlock = helpers.testBlock;
  global.saveBlockHTML = helpers.saveBlockHTML;
  global.createIframePreview = helpers.createIframePreview;
  
  console.log('✓ Loaded helper functions from scripts/ipynb-helpers.js');
  console.log('✓ Available functions:');
  console.log('  - global.testBlock(blockName, innerHTML)');
  console.log('  - global.saveBlockHTML(blockName, innerHTML, filename, options)');
  console.log('  - global.createIframePreview(blockName, blockHTML)');
  console.log('  - global.loadBlockStyles(blockName)');
}

// ============================================================================
// BROWSER SETUP
// ============================================================================
else if (isBrowser) {
  // Load helpers and initialize browser environment
  const helpers = await import('./scripts/ipynb-helpers.js');
  helpers.setupBrowserEnvironment();
}

console.log('\n========================================');
console.log('Setup complete! Ready to test EDS blocks');
console.log('========================================\n');

return 'Setup complete!';
})();

## Part 1: Simple Tests

Test basic DOM functionality that works in both environments.

In [ ]:
// Context-aware DOM test using global flags
(async () => {
const doc = isNode ? global.document : document;

const testDiv = doc.createElement('div');
testDiv.textContent = 'Hello from ' + (isNode ? 'Node.js (JSLab)' : 'Browser (ipynb-viewer)') + '!';
testDiv.className = 'test';
testDiv.style.cssText = 'padding: 20px; background: #e3f2fd; border-radius: 8px; color: #1976d2; font-weight: bold;';

console.log('✓ Created element:', testDiv.outerHTML);

// Return for display
return testDiv.outerHTML;
})();

## Part 2: Testing Blocks with Content

Test blocks that require specific content structures.

In [ ]:
// Context-aware block testing using global flags
(async () => {
const testBlockFn = isNode ? global.testBlock : window.testBlock;

// Example 1: Test HelloWorld block (no content needed)
const helloBlock = await testBlockFn('helloworld');
console.log('✓ HelloWorld block created');

return helloBlock.outerHTML;
})();

In [ ]:
// Example 2: Test Accordion block with content using global flags
(async () => {
const testBlockFn = isNode ? global.testBlock : window.testBlock;

// Structure: Each row needs 2 divs (label and body)
const accordionContent = `
  <div>
    <div>What is EDS?</div>
    <div>Edge Delivery Services is Adobe's modern web platform for building fast, performant websites.</div>
  </div>
  <div>
    <div>How do blocks work?</div>
    <div>Blocks are JavaScript functions that decorate DOM elements and transform content structure.</div>
  </div>
  <div>
    <div>Why use JSLab?</div>
    <div>JSLab lets you experiment with blocks in a notebook environment for rapid testing and development.</div>
  </div>
  <div>
    <div>Why use ipynb-viewer?</div>
    <div>The ipynb-viewer block lets end users interact with executable notebooks directly on your EDS site.</div>
  </div>
`;

const accordionBlock = await testBlockFn('accordion', accordionContent);
console.log('✓ Accordion block created');

// Show stats
const doc = isNode ? global.document : document;
const detailsCount = accordionBlock.querySelectorAll('details').length;
console.log('Accordion sections:', detailsCount);

return accordionBlock.outerHTML;
})();

## Part 3: Visual Output with Iframe

Create visual previews with iframe controls - works in both Node.js and browser!

In [ ]:
// Context-aware iframe preview generation using global flags
(async () => {
const accordionContent = `
  <div>
    <div>What is EDS?</div>
    <div>Edge Delivery Services is Adobe's modern web platform for building fast, performant websites.</div>
  </div>
  <div>
    <div>How do blocks work?</div>
    <div>Blocks are JavaScript functions that decorate DOM elements and transform content structure.</div>
  </div>
  <div>
    <div>Why use JSLab?</div>
    <div>JSLab lets you experiment with blocks in a notebook environment for rapid testing and development.</div>
  </div>
  <div>
    <div>Why use ipynb-viewer?</div>
    <div>The ipynb-viewer block lets end users interact with executable notebooks directly on your EDS site.</div>
  </div>
`;

if (isNode) {
  // Node.js: Save files to disk
  await global.saveBlockHTML('accordion', accordionContent);
  
  console.log('\n✅ FILES CREATED (Node.js):');
  console.log('📂 ipynb-tests/accordion-preview.html - Styled block');
  console.log('📂 ipynb-tests/accordion-live-preview.html - Iframe with controls');
  console.log('🎨 Open accordion-live-preview.html in your browser!');
  
  return 'Files saved! Open accordion-live-preview.html in your browser';
} else {
  // Browser: Open in new window
  const testBlockFn = window.testBlock;
  const accordionBlock = await testBlockFn('accordion', accordionContent);
  
  // Open iframe preview
  window.openIframePreview('accordion', accordionBlock.outerHTML);
  
  console.log('\n✅ PREVIEW OPENED (Browser):');
  console.log('🎨 Iframe preview opened in new window');
  console.log('🖼️  Features: Refresh button, Close button (ESC key)');
  
  return 'Iframe preview opened in new window!';
}
})();

## Part 4: Discover Available Blocks

List all blocks available for testing in this project.

In [ ]:
// Context-aware block discovery using global flags
(async () => {
if (isNode) {
  // Node.js: Use fs to list directories
  const fs = require('fs').promises;
  const blocksDir = './blocks';
  
  const entries = await fs.readdir(blocksDir, { withFileTypes: true });
  const blocks = entries
    .filter(entry => entry.isDirectory())
    .map(entry => entry.name)
    .sort();
  
  console.log(`Found ${blocks.length} blocks:\n`);
  console.log(blocks.join('\n'));
  
  return blocks;
} else {
  // Browser: Use fetch to get block list (if available)
  console.log('⚠ Block discovery requires Node.js/JSLab environment');
  console.log('ℹ️  In browser, blocks must be specified manually');
  
  // Return common blocks
  const commonBlocks = ['accordion', 'cards', 'columns', 'fragment', 'header', 'footer', 'hero'];
  console.log('Common blocks:', commonBlocks.join(', '));
  
  return commonBlocks;
}
})();

## Part 5: Interactive Calculations

Pure JavaScript that works in both environments.

In [ ]:
// Simple calculation - works everywhere
const a = 10;
const b = 20;
const sum = a + b;

console.log('Sum:', sum);
console.log('Product:', a * b);
console.log('Average:', sum / 2);

sum

In [ ]:
// Array operations - works everywhere
const numbers = [1, 2, 3, 4, 5];
const doubled = numbers.map(n => n * 2);
const sum = numbers.reduce((acc, n) => acc + n, 0);

console.log('Original:', numbers);
console.log('Doubled:', doubled);
console.log('Sum:', sum);

doubled

## Part 6: String Manipulation

In [ ]:
// Text transformations - works everywhere
const text = 'hello world from jupyter';
const upper = text.toUpperCase();
const titleCase = text.replace(/\b\w/g, c => c.toUpperCase());

console.log('Original:', text);
console.log('Uppercase:', upper);
console.log('Title Case:', titleCase);

titleCase

## Quick Reference

### Global Environment Flags (Set by Cell 1)

After running Cell 1, these global flags are available in all subsequent cells:

**Node.js (JSLab):**
```javascript
global.isNode      // true
global.isBrowser   // false
```

**Browser (ipynb-viewer):**
```javascript
window.isNode      // false
window.isBrowser   // true
```

**Usage in cells:**
```javascript
// Simply use the global flags directly
const doc = isNode ? global.document : document;
const testBlockFn = isNode ? global.testBlock : window.testBlock;
```

**No need to re-detect environment in cells!** Cell 1 sets these flags globally.

### Context-Aware DOM Access
```javascript
const doc = isNode ? global.document : document;
const testBlockFn = isNode ? global.testBlock : window.testBlock;
```

### Browser Compatibility

**Wrap async code in IIFE:**
```javascript
(async () => {
  const block = await testBlockFn('blockname', '<div>content</div>');
  return block.outerHTML;
})();
```

### Node.js Helpers (JSLab only)

**Test blocks (returns DOM element):**
```javascript
const block = await global.testBlock('blockname', '<div>content</div>');
block.outerHTML
```

**Save with Live Preview (creates iframe wrapper):**
```javascript
// Creates TWO files:
// 1. blockname-preview.html (actual styled block)
// 2. blockname-live-preview.html (iframe wrapper with controls)
await global.saveBlockHTML('blockname', '<div>content</div>');

// Custom filename
await global.saveBlockHTML('blockname', '<div>content</div>', 'my-test.html');

// Disable live preview
await global.saveBlockHTML('blockname', '<div>content</div>', null, { livePreview: false });
```

**Create iframe preview HTML:**
```javascript
const previewHTML = global.createIframePreview('blockname', '<div>block html</div>');
```

**Load styles manually:**
```javascript
await global.loadBlockStyles('blockname');
```

### Browser Helpers (ipynb-viewer)

**Test blocks:**
```javascript
const block = await window.testBlock('blockname', '<div>content</div>');
const container = window.displayBlock(block);
```

**Create and open iframe preview:**
```javascript
// Generate iframe HTML
const previewHTML = window.createIframePreview('blockname', '<div>block html</div>');

// Open preview in new window
window.openIframePreview('blockname', '<div>block html</div>');
```

### Live Preview Features
- 🔴 Dark themed wrapper with controls
- ↻ Refresh button to reload preview
- ✕ Close button (or press ESC key)
- Status bar showing file location
- Fullscreen display with scrolling

### Tips
1. **Cell 1** auto-detects environment and sets global `isNode` and `isBrowser` flags
2. **No re-detection needed** - Just use the global flags in all cells
3. **JSLab mode**: Full block testing with jsdom, saves HTML files to disk
4. **Browser mode**: Direct DOM interaction, opens preview in new window
5. **Live preview**: Works in both environments with different mechanisms
6. **Context-aware**: Write code once, runs in both environments
7. **Node.js**: Files saved to `ipynb-tests/` directory
8. **Browser**: Preview opens in popup window via Blob URL
9. **Browser async**: Wrap `await` in `(async () => { ... })()` IIFE

### Environment Capabilities

| Feature | JSLab (Node.js) | Browser (ipynb-viewer) |
|---------|----------------|------------------------|
| DOM Creation | ✅ jsdom | ✅ Native |
| Block Testing | ✅ Full | ⚠️ Limited |
| File I/O | ✅ Yes | ❌ No |
| Live Preview | ✅ Saved files | ✅ Popup window |
| Iframe Preview | ✅ Yes | ✅ Yes |
| Calculations | ✅ Yes | ✅ Yes |
| String Ops | ✅ Yes | ✅ Yes |
| Top-level await | ✅ Yes | ❌ No (use IIFE) |
| Global Flags | ✅ global.isNode, global.isBrowser | ✅ window.isNode, window.isBrowser |